In [136]:
workspace()

In [18]:
using EvolvingGraphs
using EvolvingGraphs.Centrality
using StatsBase
using DataStructures

# Katz Motivation exmple 

In [54]:
g = EvolvingGraph{Node{String}, Int}()

Directed EvolvingGraph 0 nodes, 0 static edges, 0 timestamps

In [55]:
add_bunch_of_edges!(g, [("A", "B", 1), ("A", "C", 2), ("A", "B", 2),("C", "A", 2),("B", "C", 3)])

Directed EvolvingGraph 3 nodes, 5 static edges, 3 timestamps

In [4]:
edges(g)

5-element Array{EvolvingGraphs.WeightedTimeEdge{EvolvingGraphs.Node{String},Int64,Float64},1}:
 Node(A)-1.0->Node(B) at time 1
 Node(A)-1.0->Node(C) at time 2
 Node(A)-1.0->Node(B) at time 2
 Node(C)-1.0->Node(A) at time 2
 Node(B)-1.0->Node(C) at time 3

In [56]:
katz(g)

3-element Array{Tuple{EvolvingGraphs.Node{String},Float64},1}:
 (Node(A), 0.742301)
 (Node(B), 0.42943) 
 (Node(C), 0.514373)

In [90]:
g2 = EvolvingGraph{Node{String}, Int}()
add_bunch_of_edges!(g2, [("A", "B", 3), ("A", "C", 2), ("A", "B", 2),("C", "A", 2),("B", "C", 1)])

Directed EvolvingGraph 3 nodes, 5 static edges, 3 timestamps

In [5]:
katz(g2)

3-element Array{Tuple{EvolvingGraphs.Node{String},Float64},1}:
 (Node(A), 0.687679)
 (Node(B), 0.490062)
 (Node(C), 0.535666)

In [187]:
g3 = EvolvingGraph{Node{String}, Int}()
add_bunch_of_edges!(g3, [("A", "B", 100), ("A", "C", 10), ("A", "B", 10),("C", "A", 10),("B", "C", 1)])
katz(g3)

3-element Array{Tuple{EvolvingGraphs.Node{String},Float64},1}:
 (Node(A), 0.687679)
 (Node(B), 0.490062)
 (Node(C), 0.535666)

In [7]:
g1 = EvolvingGraph{Node{String}, Int}()
add_bunch_of_edges!(g1, [("A","B", 1),("B","C", 2)])
katz(g1)

3-element Array{Tuple{EvolvingGraphs.Node{String},Float64},1}:
 (Node(A), 0.64654) 
 (Node(B), 0.604677)
 (Node(C), 0.465136)

In [146]:
g2 = EvolvingGraph{Node{String}, Int}()
add_bunch_of_edges!(g2, [("A", "B", 3), ("A", "C", 2), ("A", "B", 2),("C", "A", 2),("B", "C", 1)])
katz(g2)

3-element Array{Tuple{EvolvingGraphs.Node{String},Float64},1}:
 (Node(A), 0.687679)
 (Node(B), 0.490062)
 (Node(C), 0.535666)

# Katz Centrality

Each node `v` has forword neighbours. Each forward neighbour has forword neighbour, and so on.

In [30]:
"""
distance between two active nodes

alpha: time unit parameter
"""
function temporal_distance(v1::TimeNode, v2::TimeNode, beta::Real = 0.0)
    return beta* abs(node_timestamp(v1) - node_timestamp(v2))
end
function temporal_distance(v1::Tuple{Int,Int}, v2::Tuple{Int,Int}, beta::Real = 0.0)
    return beta * abs(v1[2] - v2[2])
end


temporal_distance (generic function with 4 methods)

In [26]:
function temporal_katz(g::AbstractEvolvingGraph, start::TimeNode; alpha = 0.1, k = 10)
    score = 0.
    v = start
    fronter = [v]
    level = 0
    while level < k
        next = []
        for u in fronter
            for v in forward_neighbors(g, u)
                push!(next, v)
                if node_key(v) != node_key(u)    
                    td = temporal_distance(start, u)
                    d = td + level
                    score += alpha^d
                end
            end
        end
        fronter = next
        level += 1
    end
    return score / num_edges(g)
end

function temporal_katz(g::IntAdjacencyList, start::Tuple{Int,Int}; alpha = 0.1, k = 10)
    score = 0.
    v = start
    fronter = [v]
    level = 0
    while level < k
        next = []
        for u in fronter
            fn = forward_neighbors(g, u)
            for v in fn
                push!(next, v)
                td = temporal_distance(start, u)
                d = td + level
                score += alpha^d
            end
        end
        fronter = next
        level += 1
    end
    return score/ num_edges(g)
end

# Can be done in parallel, focus on useful node at certain time stamp


temporal_katz (generic function with 2 methods)

In [82]:
active_nodes(g2)

7-element Array{EvolvingGraphs.TimeNode{String,Int64},1}:
 TimeNode(A, 3)
 TimeNode(B, 3)
 TimeNode(A, 2)
 TimeNode(C, 2)
 TimeNode(B, 2)
 TimeNode(B, 1)
 TimeNode(C, 1)

In [83]:
an = active_nodes(g2)

scores = Dict()
for n in an
    scores[n] = temporal_katz(g2, n, alpha = 0.2, k = 10)
end
scores

Dict{Any,Any} with 7 entries:
  TimeNode(C, 2) => 0.293333
  TimeNode(B, 2) => 0.0
  TimeNode(A, 3) => 0.2
  TimeNode(A, 2) => 0.466667
  TimeNode(B, 3) => 0.0
  TimeNode(B, 1) => 0.202347
  TimeNode(C, 1) => 0.0117333

In [153]:
using DataStructures

sum_scores = DefaultDict(0.) 
for (n, v) in scores
    sum_scores[node_key(n)] += v
end
sum_scores 

DataStructures.DefaultDict{Any,Any,Float64} with 3 entries:
  "B" => 1.41227
  "A" => 0.471678
  "C" => 0.106402

In [76]:
an = active_nodes(g3)

scores = Dict()
for n in an
    scores[n] = temporal_katz(g3, n, alpha = 0.2, k = 10)
end
scores

Dict{Any,Any} with 7 entries:
  TimeNode(B, 10)  => 0.0
  TimeNode(B, 100) => 0.0
  TimeNode(A, 100) => 0.2
  TimeNode(C, 10)  => 0.291667
  TimeNode(A, 10)  => 0.458333
  TimeNode(C, 1)   => 2.98666e-8
  TimeNode(B, 1)   => 0.2

In [26]:
an = active_nodes(g3)

scores = Dict()
for n in an
    scores[n] = temporal_katz(g3, n, alpha = 0.2, k = 10)
end
scores

Dict{Any,Any} with 7 entries:
  TimeNode(C, 1)   => 1.06833
  TimeNode(B, 100) => 0.0
  TimeNode(C, 2)   => 1.70833
  TimeNode(B, 2)   => 1.0
  TimeNode(B, 1)   => 3.25367
  TimeNode(A, 100) => 1.0
  TimeNode(A, 2)   => 3.54167

# Communicability Betweenness Centrality 

In [375]:
"""
Take account of all forward neighours.
"""

function temporal_katz(g::AbstractEvolvingGraph, start_node::TimeNode, end_node::TimeNode; alpha = 0.2, k = 10)
    score = 0.
    v = start_node
    fronter = [v]
    level = 0
    while level < k
        next = []
        for u in fronter
            for v in forward_neighbors(g, u)
                push!(next, v)
                if u == end_node
                    if node_key(v) != node_key(u) 
                        td = temporal_distance(start_node, u)
                        d = td + level
                        score += alpha^d
                    end
                end
            end
        end
        fronter = next
        level += 1
    end
    return score
end


function temporal_resolvent_betweenness(g1::EvolvingGraph, g2::EvolvingGraph, v::TimeNode; alpha = 0.2, k = 10)
    r = 0.
    ns = active_nodes(g1)
    for i in ns
        for j in ns
            if i != j && i != v && j != v

                score1 = temporal_katz(g1, i, j, alpha = alpha, k = k)
                score2 = temporal_katz(g2, i, j, alpha = alpha, k = k)
#                 println("From node $i to node $j")
#                 println("score1 $score1 -> score2 $score2")
                if score1 != 0.0
                    r += (score1 - score2)/score1
                end
            end
        end
    end
    return r
end

temporal_resolvent_betweenness (generic function with 1 method)

In [116]:
g1 = EvolvingGraph{Node{String}, Int}()
add_bunch_of_edges!(g1, [("A", "B", 3), ("A", "C", 2), ("A", "B", 2),("C", "A", 2),("B", "C", 1)])

g2 = EvolvingGraph{Node{String}, Int}()
add_bunch_of_edges!(g2, [("A", "C", 2), ("A", "B", 2),("C", "A", 2),("B", "C", 1)])

v = active_nodes(g1)[2]

TimeNode(B, 3)

In [117]:
temporal_resolvent_betweenness(g1, g2, v; alpha = 0.2, k = 10)

4.0

In [122]:
g1 = EvolvingGraph{Node{String}, Int}()
add_bunch_of_edges!(g1, [("A", "B", 3), ("A", "C", 2), ("A", "B", 2),("C", "A", 2),("B", "C", 1)])

g2 = EvolvingGraph{Node{String}, Int}()
add_bunch_of_edges!(g2, [("A", "B", 3),("B", "C", 1)])

v = active_nodes(g1)[3]

TimeNode(A, 2)

In [123]:
temporal_resolvent_betweenness(g1, g2, v; alpha = 0.2, k = 10)

5.0

In [89]:
v = active_nodes(g1)[2]
println(v)
temporal_resolvent_betweenness(g1, g2, v; alpha = 0.2, k = 10)

TimeNode(B, 100)


9.0

In [14]:
an = active_nodes(g)

scores = Dict()
for n in an
    scores[n] = temporal_communicability(g, n, alpha = 0.1, k = 100)
end
scores

Dict{Any,Any} with 7 entries:
  TimeNode(B, 3) => 0.0
  TimeNode(A, 2) => 0.0564789
  TimeNode(C, 3) => 0
  TimeNode(A, 1) => 0.0
  TimeNode(B, 2) => 0.0
  TimeNode(B, 1) => 0.0
  TimeNode(C, 2) => 0.0860369

In [12]:
0/0

NaN

# Temporal Betweenness

In constrast with Katz and Communicability (which are based on walks), the following centrality algorithms are based on shortest paths.

In [376]:
#TODO later

# Temporal Closeness Centrality 

Closeness centrality (or closeness) of a node is a measure of centrality in a network, calculated as the sum of the length of the shortest paths between the node and all other nodes in the graph.

In [29]:
function temporal_closeness(g::Union{AbstractEvolvingGraph, IntAdjacencyList}, start::Union{TimeNode,Tuple{Int,Int}})
    v = start
    level = Dict(v => 0)
    i = 1
    fronter = [v]
    while length(fronter) > 0
        next = []
        for u in fronter
            for v in forward_neighbors(g, u)
                if !(v in keys(level))
                    if node_key(u) != node_key(v)
                        td = temporal_distance(start, u)
                        level[v] = i + td
                    else
                        level[v] = i - 1
                    end
                    push!(next, v)
                end
            end
        end
        fronter = next
        i += 1
    end
    
    total_scores = sum(values(level))
    return total_scores > 0. ? (length(level) - 1)/total_scores : 0.
end

temporal_closeness (generic function with 1 method)

In [2]:
function temporal_closeness(g::IntAdjacencyList, start::Tuple{Int,Int})
    v = start
    level = Dict(v => 0)
    i = 1
    fronter = [v]
    while length(fronter) > 0
        next = []
        for u in fronter
            for v in forward_neighbors(g, u)
                if !(v in keys(level))
                    if u[1] != v[1]
                        td = temporal_distance(start, u)
                        level[v] = i + td
                    else
                        level[v] = i - 1
                    end
                    push!(next, v)
                end
            end
        end
        fronter = next
        i += 1
    end
    
    total_scores = sum(values(level))
    return total_scores > 0. ? (length(level) - 1)/total_scores : 0.
end

temporal_closeness (generic function with 1 method)

In [125]:
edges(g)

5-element Array{EvolvingGraphs.WeightedTimeEdge{EvolvingGraphs.Node{String},Int64,Float64},1}:
 Node(A)-1.0->Node(B) at time 1
 Node(A)-1.0->Node(C) at time 2
 Node(A)-1.0->Node(B) at time 2
 Node(C)-1.0->Node(A) at time 2
 Node(B)-1.0->Node(C) at time 3

In [131]:
an = active_nodes(g2)

scores = Dict()
for n in an
    scores[n] = temporal_closeness(g2, n)
end
scores

Dict{Any,Any} with 7 entries:
  TimeNode(B, 2) => 0.0
  TimeNode(A, 3) => 1.0
  TimeNode(B, 1) => 0.346154
  TimeNode(B, 3) => 0.0
  TimeNode(A, 2) => 0.666667
  TimeNode(C, 1) => 0.315789
  TimeNode(C, 2) => 0.5

In [133]:
an = active_nodes(g3)

scores = Dict()
for n in an
    scores[n] = temporal_closeness(g3, n)
end
scores

Dict{Any,Any} with 7 entries:
  TimeNode(B, 10)  => 0.0
  TimeNode(B, 100) => 0.0
  TimeNode(A, 100) => 1.0
  TimeNode(C, 10)  => 0.0412371
  TimeNode(A, 10)  => 0.0707071
  TimeNode(C, 1)   => 0.0390625
  TimeNode(B, 1)   => 0.0597015

# Temporal PageRank

Random walk interpretation can not travel back in time.

* Block matrix version (use reverse pagerank): useful for evolving graph with a small number of timestamps

* Probability based (time-stamps based rating): Random walk interpretation. Not fixed probability for all nodes but different probability for each individual node. 

In [30]:
function block_google_matrix(g::IntAdjacencyList; alpha = 0.85, balance = 1)
    A = full(block_adjacency_matrix(g))
    M = balance * A + (1-balance) * A'
    N, N = size(M)
    p = ones(N)/N
    dangling_weights = p
    dangling_nodes = find(x->x==0, sum(M,2))
    
    for node in dangling_nodes
        M[node,:] = dangling_weights
    end
    M ./= sum(M,2)
    return alpha * M .+ (1- alpha) * p
end

block_google_matrix (generic function with 1 method)

In [31]:
g2 = EvolvingGraph{Node{String}, Int}()
add_bunch_of_edges!(g2, [("A", "B", 3), ("A", "C", 2),("B", "C", 1)])

Directed EvolvingGraph 3 nodes, 3 static edges, 3 timestamps

In [35]:
function temporal_pagerank(g::AbstractEvolvingGraph)
    A = block_google_matrix(g)
    F = eigfact(A')
    vals = abs.(F[:values])
    _, index = findmax(vals)
    return abs.(F[:vectors][:,index])
end

temporal_pagerank (generic function with 1 method)

In [36]:
node_i = Dict()
for n in nodes(g2)
    node_i[node_key(n)] = node_index(n)
end
node_i

Dict{Any,Any} with 3 entries:
  "B" => 2
  "A" => 1
  "C" => 3

In [40]:
nn = num_nodes(g)
r = temporal_pagerank(g)
ns = nodes(g)
#scores = Dict()
#for n in ns
#    t = node_timestamp(n)
#    i = node_i[node_key(n)]
#    scores[n] = r[i+(t-1)*nn]
#end
#scores

500-element Array{Int64,1}:
   1
   2
   3
   4
   5
   6
   7
   8
   9
  10
  11
  12
  13
   ⋮
 489
 490
 491
 492
 493
 494
 495
 496
 497
 498
 499
 500

In [42]:
r[1:500]

500-element Array{Float64,1}:
 0.0111377
 0.0114701
 0.011841 
 0.0121502
 0.0118863
 0.0122059
 0.0125376
 0.0139144
 0.0123537
 0.0114941
 0.0132801
 0.0108638
 0.0139721
 ⋮        
 0.0118571
 0.0116247
 0.010082 
 0.0128428
 0.010226 
 0.012166 
 0.0106454
 0.0122471
 0.0118073
 0.0119758
 0.0116085
 0.0126577

In [284]:
using DataStructures

sum_scores = DefaultDict(0.) 
for (n, v) in scores
    sum_scores[node_key(n)] += v
end
sum_scores 

DataStructures.DefaultDict{Any,Any,Float64} with 3 entries:
  "B" => 0.571454
  "A" => 0.3811
  "C" => 0.504663

# Random Evolving Graph Data

Start with the original definition and compare the change of ranking of top authors if we vary

In [3]:
srand(1234)
g = random_evolving_graph(500, 10, 0.2)

Directed IntAdjacencyList (500 nodes, 499183 static edges, 10 timestamps)

In [24]:
r = katz(g)
sorted_top = sort(r, by = x -> x[2], rev = true)[1:10]

10-element Array{Tuple{Int64,Float64},1}:
 (459, 0.126296) 
 (147, 0.116541) 
 (326, 0.113667) 
 (183, 0.108705) 
 (469, 0.0936847)
 (296, 0.0925748)
 (424, 0.0921004)
 (377, 0.091973) 
 (313, 0.0910352)
 (127, 0.089123) 

In [6]:
sorted_top_keys = map(x -> x[1], sorted_top);

In [12]:
scores_clo2 = []
for n in sorted_top_keys
    for t in 1:10
        intn = (n, t)
        tr = temporal_closeness(g, intn)
        push!(scores_clo2, (intn, tr))
    end
end
scores_clo2

100-element Array{Any,1}:
 ((459, 1), 0.491544) 
 ((459, 2), 0.488173) 
 ((459, 3), 0.493034) 
 ((459, 4), 0.48409)  
 ((459, 5), 0.492366) 
 ((459, 6), 0.501002) 
 ((459, 7), 0.498379) 
 ((459, 8), 0.496687) 
 ((459, 9), 0.502262) 
 ((459, 10), 0.555061)
 ((147, 1), 0.479015) 
 ((147, 2), 0.481279) 
 ((147, 3), 0.494375) 
 ⋮                    
 ((313, 9), 0.515214) 
 ((313, 10), 0.56257) 
 ((127, 1), 0.483135) 
 ((127, 2), 0.484754) 
 ((127, 3), 0.480014) 
 ((127, 4), 0.490331) 
 ((127, 5), 0.497594) 
 ((127, 6), 0.496523) 
 ((127, 7), 0.493946) 
 ((127, 8), 0.499334) 
 ((127, 9), 0.507107) 
 ((127, 10), 0.55692) 

In [37]:
time_scores = []
for (n, v) in scores_clo
    if n[2] == 1
        push!(time_scores, (n, v))
    end
end
sort(time_scores, by = x -> x[2], rev = true)

10-element Array{Any,1}:
 ((296, 1), 0.253937)
 ((469, 1), 0.248744)
 ((377, 1), 0.247757)
 ((459, 1), 0.247512)
 ((424, 1), 0.247133)
 ((326, 1), 0.247059)
 ((127, 1), 0.244593)
 ((183, 1), 0.242235)
 ((313, 1), 0.238867)
 ((147, 1), 0.237313)

In [19]:
sum_scores = DefaultDict(0.) 
for (n, v) in scores_clo
    sum_scores[n[1]] += v
end
sum_scores
sort(collect(sum_scores), by = x -> x[2], rev = true)

10-element Array{Pair{Any,Any},1}:
 Pair{Any,Any}(469, 3.5618) 
 Pair{Any,Any}(147, 3.55717)
 Pair{Any,Any}(183, 3.55485)
 Pair{Any,Any}(313, 3.54834)
 Pair{Any,Any}(377, 3.54467)
 Pair{Any,Any}(296, 3.54125)
 Pair{Any,Any}(459, 3.54034)
 Pair{Any,Any}(127, 3.53797)
 Pair{Any,Any}(424, 3.5353) 
 Pair{Any,Any}(326, 3.53318)

In [39]:
sum_scores = DefaultDict(0.) 
for (n, v) in scores_clo2
    sum_scores[n[1]] += v
end
sum_scores
sort(collect(sum_scores), by = x -> x[2], rev = true)

10-element Array{Pair{Any,Any},1}:
 Pair{Any,Any}(469, 5.04016)
 Pair{Any,Any}(183, 5.02977)
 Pair{Any,Any}(313, 5.01658)
 Pair{Any,Any}(326, 5.00902)
 Pair{Any,Any}(147, 5.00863)
 Pair{Any,Any}(459, 5.0026) 
 Pair{Any,Any}(296, 5.00257)
 Pair{Any,Any}(377, 4.9969) 
 Pair{Any,Any}(127, 4.98966)
 Pair{Any,Any}(424, 4.98694)

In [38]:
time_scores = []
for (n, v) in scores_clo2
    if n[2] == 1
        push!(time_scores, (n, v))
    end
end
sort(time_scores, by = x -> x[2], rev = true)

10-element Array{Any,1}:
 ((296, 1), 0.493826)
 ((469, 1), 0.493387)
 ((459, 1), 0.491544)
 ((424, 1), 0.488756)
 ((326, 1), 0.487898)
 ((377, 1), 0.487184)
 ((183, 1), 0.485764)
 ((127, 1), 0.483135)
 ((147, 1), 0.479015)
 ((313, 1), 0.477277)

In [31]:
scores_2 = []

for n in sorted_top_keys
    for t in 1:10
        intn = (n, t)
        tr = temporal_katz(g, intn, alpha = 0.5, k = 3)
        push!(scores_2, (intn, tr))
    end
end
scores

100-element Array{Any,1}:
 ((459, 1), 0.62903)  
 ((459, 2), 0.579759) 
 ((459, 3), 0.563742) 
 ((459, 4), 0.522358) 
 ((459, 5), 0.548133) 
 ((459, 6), 0.599635) 
 ((459, 7), 0.541802) 
 ((459, 8), 0.47214)  
 ((459, 9), 0.441643) 
 ((459, 10), 0.502562)
 ((147, 1), 0.547159) 
 ((147, 2), 0.49845)  
 ((147, 3), 0.601793) 
 ⋮                    
 ((313, 9), 0.513323) 
 ((313, 10), 0.557448)
 ((127, 1), 0.549244) 
 ((127, 2), 0.553146) 
 ((127, 3), 0.46508)  
 ((127, 4), 0.524687) 
 ((127, 5), 0.606209) 
 ((127, 6), 0.554106) 
 ((127, 7), 0.528815) 
 ((127, 8), 0.503844) 
 ((127, 9), 0.436141) 
 ((127, 10), 0.515292)

In [33]:
time_scores = []
for (n, v) in scores_2
    if n[2] == 1
        push!(time_scores, (n, v))
    end
end
sort(time_scores, by = x -> x[2], rev = true)

10-element Array{Any,1}:
 ((469, 1), 0.7193)  
 ((459, 1), 0.718027)
 ((424, 1), 0.710843)
 ((296, 1), 0.704199)
 ((326, 1), 0.685345)
 ((183, 1), 0.66474) 
 ((377, 1), 0.66221) 
 ((127, 1), 0.631586)
 ((147, 1), 0.630289)
 ((313, 1), 0.571531)

In [24]:
time_scores = []
for (n, v) in scores_2
    if n[2] == 1
        push!(time_scores, (n, v))
    end
end
sort(time_scores, by = x -> x[2], rev = true)

10-element Array{Any,1}:
 ((459, 1), 0.62903) 
 ((469, 1), 0.62809) 
 ((424, 1), 0.623551)
 ((296, 1), 0.616669)
 ((326, 1), 0.598365)
 ((183, 1), 0.577783)
 ((377, 1), 0.577505)
 ((127, 1), 0.549244)
 ((147, 1), 0.547159)
 ((313, 1), 0.491694)

In [35]:
time_scores = []
for (n, v) in scores
    if n[2] == 1
        push!(time_scores, (n, v))
    end
end
sort(time_scores, by = x -> x[2], rev = true)

10-element Array{Any,1}:
 ((459, 1), 0.62903) 
 ((469, 1), 0.62809) 
 ((424, 1), 0.623551)
 ((296, 1), 0.616669)
 ((326, 1), 0.598365)
 ((183, 1), 0.577783)
 ((377, 1), 0.577505)
 ((127, 1), 0.549244)
 ((147, 1), 0.547159)
 ((313, 1), 0.491694)

In [36]:
sum_scores = DefaultDict(0.) 
for (n, v) in scores
    sum_scores[n[1]] += v
end
sort(collect(sum_scores), by = x -> x[2], rev = true)

10-element Array{Pair{Any,Any},1}:
 Pair{Any,Any}(183, 5.5761) 
 Pair{Any,Any}(469, 5.53313)
 Pair{Any,Any}(326, 5.41495)
 Pair{Any,Any}(313, 5.4089) 
 Pair{Any,Any}(459, 5.4008) 
 Pair{Any,Any}(147, 5.3966) 
 Pair{Any,Any}(377, 5.31166)
 Pair{Any,Any}(296, 5.29904)
 Pair{Any,Any}(424, 5.25772)
 Pair{Any,Any}(127, 5.23656)

In [26]:
using DataStructures

sum_scores = DefaultDict(0.) 
for (n, v) in scores_2
    sum_scores[n[1]] += v
end
sum_scores
sort(collect(sum_scores), by = x -> x[2], rev = true)

10-element Array{Pair{Any,Any},1}:
 Pair{Any,Any}(183, 5.5761) 
 Pair{Any,Any}(469, 5.53313)
 Pair{Any,Any}(326, 5.41495)
 Pair{Any,Any}(313, 5.4089) 
 Pair{Any,Any}(459, 5.4008) 
 Pair{Any,Any}(147, 5.3966) 
 Pair{Any,Any}(377, 5.31166)
 Pair{Any,Any}(296, 5.29904)
 Pair{Any,Any}(424, 5.25772)
 Pair{Any,Any}(127, 5.23656)

In [42]:
using DataStructures

sum_scores = DefaultDict(0.) 
for (n, v) in scores
    sum_scores[n] += v
end
sum_scores
sort(collect(sum_scores), by = x -> x[2], rev = true)

10-element Array{Pair{Any,Any},1}:
 Pair{Any,Any}((414, 5), 0.718281)
 Pair{Any,Any}((334, 5), 0.714431)
 Pair{Any,Any}((35, 5), 0.71414)  
 Pair{Any,Any}((9, 5), 0.713123)  
 Pair{Any,Any}((123, 5), 0.711902)
 Pair{Any,Any}((428, 5), 0.711783)
 Pair{Any,Any}((478, 5), 0.711754)
 Pair{Any,Any}((164, 5), 0.711648)
 Pair{Any,Any}((291, 5), 0.695053)
 Pair{Any,Any}((250, 5), 0.657184)

In [31]:
accu_scores = DefaultDict([0.]) 
for (n, v) in scores
#     println("node $n")
#     println("value $v")
    push!(accu_scores[node_key(n)], v)
#     push!(accu_scores[node_key(n)], accu_scores[node_key(n)][end] + v)
end
accu_scores

node TimeNode(11, 1)
value 1224.834899819936
node TimeNode(17, 1)
value 1163.8518998292761
node TimeNode(45, 1)
value 1293.5838998093886
node TimeNode(52, 1)
value 906.0639998690834
node TimeNode(25, 1)
value 778.2052998885803
node TimeNode(34, 1)
value 1238.168899817753
node TimeNode(36, 1)
value 1539.8606997719116
node TimeNode(54, 1)
value 1144.227099832334
node TimeNode(55, 1)
value 1169.1183998284694
node TimeNode(43, 1)
value 1099.1326998390223
node TimeNode(80, 1)
value 1363.896299798577
node TimeNode(23, 1)
value 1047.9910998470273
node TimeNode(6, 1)
value 1296.9582998088517
node TimeNode(37, 1)
value 1228.5242998193146
node TimeNode(46, 1)
value 910.8328998682736
node TimeNode(60, 1)
value 1141.6199998327547
node TimeNode(89, 1)
value 1341.6504998016464
node TimeNode(33, 1)
value 922.5920998664808
node TimeNode(73, 1)
value 900.4506998700047
node TimeNode(94, 1)
value 1128.5436998347964
node TimeNode(11, 2)
value 746.1284998952133
node TimeNode(17, 2)
value 910.4348998706787


DataStructures.DefaultDict{Any,Any,Array{Float64,1}} with 0 entries

In [30]:
using Plots

INFO: Precompiling module Plots.


In [26]:
r2 = collect(sum_scores);

In [27]:
sorted_r2 = sort(r2, by = x -> x[2], rev = true)

20-element Array{Pair{Any,Any},1}:
 Pair{Any,Any}(2, 3596.32)  
 Pair{Any,Any}(58, 3370.67) 
 Pair{Any,Any}(7, 3226.47)  
 Pair{Any,Any}(11, 3048.78) 
 Pair{Any,Any}(88, 3047.22) 
 Pair{Any,Any}(18, 2958.37) 
 Pair{Any,Any}(57, 2957.42) 
 Pair{Any,Any}(65, 2942.76) 
 Pair{Any,Any}(32, 2939.11) 
 Pair{Any,Any}(97, 2930.67) 
 Pair{Any,Any}(33, 2926.23) 
 Pair{Any,Any}(45, 2879.59) 
 Pair{Any,Any}(87, 2832.12) 
 Pair{Any,Any}(10, 2771.23) 
 Pair{Any,Any}(100, 2756.39)
 Pair{Any,Any}(68, 2746.95) 
 Pair{Any,Any}(44, 2738.5)  
 Pair{Any,Any}(83, 2672.8)  
 Pair{Any,Any}(74, 2669.7)  
 Pair{Any,Any}(26, 2452.59) 

In [28]:
sorted_top

20-element Array{Tuple{EvolvingGraphs.Node{Int64},Float64},1}:
 (Node(32), 0.320363) 
 (Node(58), 0.244402) 
 (Node(100), 0.235876)
 (Node(2), 0.199925)  
 (Node(74), 0.198615) 
 (Node(83), 0.162692) 
 (Node(26), 0.152276) 
 (Node(97), 0.138456) 
 (Node(68), 0.137589) 
 (Node(18), 0.126655) 
 (Node(7), 0.120336)  
 (Node(57), 0.108358) 
 (Node(10), 0.103718) 
 (Node(44), 0.099909) 
 (Node(45), 0.08922)  
 (Node(87), 0.0857851)
 (Node(33), 0.0857679)
 (Node(88), 0.0825459)
 (Node(11), 0.0791009)
 (Node(65), 0.0738873)

In [43]:
using EvolvingGraphs
using EvolvingGraphs.Centrality
import EzXML


# load graph data at each year
# form evolving graph
g = EvolvingGraph{Node{String}, Int}()

for year in range(2001, 17)
    sg = load_graphml("jmlr_$(year).graphml")
    add_graph!(g, sg, year)
end
g

Directed EvolvingGraph 3354 nodes, 15654 static edges, 17 timestamps

In [50]:
rating = katz(g, 0.2)


sorted_authors = sort(rating, by = x -> x[2], rev = true)

println("Top 10 rating authors")
sorted_authors[1:10]
nodes(g)

Top 10 rating authors


3354-element Array{EvolvingGraphs.Node{String},1}:
 Node(Manevitz, Larry M.)      
 Node(Yousef, Malik)           
 Node(Gates, Kevin E.)         
 Node(Masters, Annette)        
 Node(Downs, Tom)              
 Node(Vapnik, Vladimir)        
 Node(Siegelmann, Hava T.)     
 Node(Ben-Hur, Asa)            
 Node(Horn, David)             
 Node(Singer, Yoram)           
 Node(Crammer, Koby)           
 Node(Genton, Marc G.)         
 Node(Pekalska, Elzbieta)      
 ⋮                             
 Node(Paul Rochet)             
 Node(Hanwen Huang)            
 Node(Niharika Challapalli)    
 Node(Mathukumalli Vidyasagar) 
 Node(Deepayan Chakrabarti)    
 Node(Sofus A. Macskassy)      
 Node(Jonathan Chang)          
 Node(Stanislav Funiak)        
 Node(Federico Ricci-Tersenghi)
 Node(Jack Raymond)            
 Node(Martin S. Copenhaver)    
 Node(Dimitris Bertsimas)      

In [ ]:
scores = []

for n in 1:10
    for t in 1:10
        intn = (n, 5)
        tr = temporal_katz(g, intn, alpha = 0.2, k = 3)
        push!(scores, (intn, tr))
    end
end
scores